In [2]:
import pandas as pd

from itertools import combinations
from gurobipy import *

from c_plot import *
from clean_data_cor import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# INSTANCIAS
# {start} chamar dados
demanda = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia1/demanda.csv')
preco_ = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia1/preco.csv')
rota0 = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia1/rota.csv')
# {end} chamar dados

rota1 = eval(rota0['Route'][0])

rota = [0] + rota1
# periodo_lim = demanda['DBD'].tolist()[-4:]
# demanda_lim = demanda[demanda['DBD'].isin(periodo_lim)]
I, J, OD, V,  T, stations, VK, P, d = clean_data(demanda, preco_)

AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

I = [i for i in rota if i in I]
I2 = [0] + I
J = [i for i in rota if i in J]
n = len(rota)-1
Q = 565 #561 #76

In [ ]:
# import random 
# rota1 = ['uno','dos','tres', 'cuatro', 'cinco']
# rota = [0] + rota1

# OD = [('uno','dos'),
#            ('uno','tres'),
#            ('uno','cuatro'),
#            ('uno','cinco'),
#            ('dos','tres'),
#            ('dos','cuatro'),
#            ('dos','cinco'),
#            ('tres','cuatro'),
#            ('tres','cinco'),
#            ('cuatro','cinco'),]

# AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
# NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

# I = ['uno','dos','tres', 'cuatro']
# I2 = [0] + I

# J = ['dos','tres','cuatro', 'cinco']
# V = ['p','z']
# T = [0,1]
# Q = 700 
# n = len(rota) - 1 
# VK = {'p':[1,5,7],'z':[1,3,7]}

# d = {(i,j,v,k,t): np.random.randint(1,100) for i,j in OD for v in V for k in VK[v] for t in T }

# P = {}
# for i,j in OD:
#     for v in V:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(VK[v])), reverse=True)
#         for k,valor in enumerate(aleat):
#             P[(i,j,v,VK[v][k])] = valor

# # corrigir instancias
# d = {(i,j,v,k,t): d[(i,j,v,k,t)] if (i,j,v,k,t) in d else 0  for i,j in OD for v in V for k in VK[v] for t in T}
# P = {(i,j,v,k): P[(i,j,v,k)] if (i,j,v,k) in P else 0  for i,j in OD for v in V for k in VK[v] }

In [ ]:
# d = {('uno', 'dos', 'p', 1, 0): 99,
#  ('uno', 'dos', 'p', 1, 1): 72,
#  ('uno', 'dos', 'p', 5, 0): 81,
#  ('uno', 'dos', 'p', 5, 1): 77,
#  ('uno', 'dos', 'p', 7, 0): 64,
#  ('uno', 'dos', 'p', 7, 1): 67,
#  ('uno', 'dos', 'z', 1, 0): 3,
#  ('uno', 'dos', 'z', 1, 1): 7,
#  ('uno', 'dos', 'z', 3, 0): 44,
#  ('uno', 'dos', 'z', 3, 1): 3,
#  ('uno', 'dos', 'z', 7, 0): 34,
#  ('uno', 'dos', 'z', 7, 1): 64,
#  ('uno', 'tres', 'p', 1, 0): 90,
#  ('uno', 'tres', 'p', 1, 1): 20,
#  ('uno', 'tres', 'p', 5, 0): 11,
#  ('uno', 'tres', 'p', 5, 1): 51,
#  ('uno', 'tres', 'p', 7, 0): 11,
#  ('uno', 'tres', 'p', 7, 1): 77,
#  ('uno', 'tres', 'z', 1, 0): 31,
#  ('uno', 'tres', 'z', 1, 1): 98,
#  ('uno', 'tres', 'z', 3, 0): 81,
#  ('uno', 'tres', 'z', 3, 1): 21,
#  ('uno', 'tres', 'z', 7, 0): 56,
#  ('uno', 'tres', 'z', 7, 1): 39,
#  ('uno', 'cuatro', 'p', 1, 0): 88,
#  ('uno', 'cuatro', 'p', 1, 1): 76,
#  ('uno', 'cuatro', 'p', 5, 0): 82,
#  ('uno', 'cuatro', 'p', 5, 1): 34,
#  ('uno', 'cuatro', 'p', 7, 0): 67,
#  ('uno', 'cuatro', 'p', 7, 1): 41,
#  ('uno', 'cuatro', 'z', 1, 0): 1,
#  ('uno', 'cuatro', 'z', 1, 1): 66,
#  ('uno', 'cuatro', 'z', 3, 0): 62,
#  ('uno', 'cuatro', 'z', 3, 1): 24,
#  ('uno', 'cuatro', 'z', 7, 0): 19,
#  ('uno', 'cuatro', 'z', 7, 1): 35,
#  ('uno', 'cinco', 'p', 1, 0): 71,
#  ('uno', 'cinco', 'p', 1, 1): 86,
#  ('uno', 'cinco', 'p', 5, 0): 26,
#  ('uno', 'cinco', 'p', 5, 1): 75,
#  ('uno', 'cinco', 'p', 7, 0): 2,
#  ('uno', 'cinco', 'p', 7, 1): 23,
#  ('uno', 'cinco', 'z', 1, 0): 59,
#  ('uno', 'cinco', 'z', 1, 1): 59,
#  ('uno', 'cinco', 'z', 3, 0): 1,
#  ('uno', 'cinco', 'z', 3, 1): 82,
#  ('uno', 'cinco', 'z', 7, 0): 3,
#  ('uno', 'cinco', 'z', 7, 1): 90,
#  ('dos', 'tres', 'p', 1, 0): 10,
#  ('dos', 'tres', 'p', 1, 1): 40,
#  ('dos', 'tres', 'p', 5, 0): 65,
#  ('dos', 'tres', 'p', 5, 1): 39,
#  ('dos', 'tres', 'p', 7, 0): 59,
#  ('dos', 'tres', 'p', 7, 1): 29,
#  ('dos', 'tres', 'z', 1, 0): 32,
#  ('dos', 'tres', 'z', 1, 1): 24,
#  ('dos', 'tres', 'z', 3, 0): 59,
#  ('dos', 'tres', 'z', 3, 1): 48,
#  ('dos', 'tres', 'z', 7, 0): 28,
#  ('dos', 'tres', 'z', 7, 1): 95,
#  ('dos', 'cuatro', 'p', 1, 0): 8,
#  ('dos', 'cuatro', 'p', 1, 1): 85,
#  ('dos', 'cuatro', 'p', 5, 0): 55,
#  ('dos', 'cuatro', 'p', 5, 1): 11,
#  ('dos', 'cuatro', 'p', 7, 0): 33,
#  ('dos', 'cuatro', 'p', 7, 1): 43,
#  ('dos', 'cuatro', 'z', 1, 0): 83,
#  ('dos', 'cuatro', 'z', 1, 1): 56,
#  ('dos', 'cuatro', 'z', 3, 0): 21,
#  ('dos', 'cuatro', 'z', 3, 1): 79,
#  ('dos', 'cuatro', 'z', 7, 0): 13,
#  ('dos', 'cuatro', 'z', 7, 1): 37,
#  ('dos', 'cinco', 'p', 1, 0): 28,
#  ('dos', 'cinco', 'p', 1, 1): 24,
#  ('dos', 'cinco', 'p', 5, 0): 39,
#  ('dos', 'cinco', 'p', 5, 1): 75,
#  ('dos', 'cinco', 'p', 7, 0): 15,
#  ('dos', 'cinco', 'p', 7, 1): 10,
#  ('dos', 'cinco', 'z', 1, 0): 30,
#  ('dos', 'cinco', 'z', 1, 1): 91,
#  ('dos', 'cinco', 'z', 3, 0): 48,
#  ('dos', 'cinco', 'z', 3, 1): 26,
#  ('dos', 'cinco', 'z', 7, 0): 2,
#  ('dos', 'cinco', 'z', 7, 1): 30,
#  ('tres', 'cuatro', 'p', 1, 0): 87,
#  ('tres', 'cuatro', 'p', 1, 1): 1,
#  ('tres', 'cuatro', 'p', 5, 0): 29,
#  ('tres', 'cuatro', 'p', 5, 1): 77,
#  ('tres', 'cuatro', 'p', 7, 0): 17,
#  ('tres', 'cuatro', 'p', 7, 1): 51,
#  ('tres', 'cuatro', 'z', 1, 0): 28,
#  ('tres', 'cuatro', 'z', 1, 1): 46,
#  ('tres', 'cuatro', 'z', 3, 0): 94,
#  ('tres', 'cuatro', 'z', 3, 1): 91,
#  ('tres', 'cuatro', 'z', 7, 0): 93,
#  ('tres', 'cuatro', 'z', 7, 1): 99,
#  ('tres', 'cinco', 'p', 1, 0): 2,
#  ('tres', 'cinco', 'p', 1, 1): 80,
#  ('tres', 'cinco', 'p', 5, 0): 3,
#  ('tres', 'cinco', 'p', 5, 1): 21,
#  ('tres', 'cinco', 'p', 7, 0): 17,
#  ('tres', 'cinco', 'p', 7, 1): 36,
#  ('tres', 'cinco', 'z', 1, 0): 88,
#  ('tres', 'cinco', 'z', 1, 1): 2,
#  ('tres', 'cinco', 'z', 3, 0): 27,
#  ('tres', 'cinco', 'z', 3, 1): 22,
#  ('tres', 'cinco', 'z', 7, 0): 47,
#  ('tres', 'cinco', 'z', 7, 1): 14,
#  ('cuatro', 'cinco', 'p', 1, 0): 4,
#  ('cuatro', 'cinco', 'p', 1, 1): 80,
#  ('cuatro', 'cinco', 'p', 5, 0): 32,
#  ('cuatro', 'cinco', 'p', 5, 1): 73,
#  ('cuatro', 'cinco', 'p', 7, 0): 64,
#  ('cuatro', 'cinco', 'p', 7, 1): 27,
#  ('cuatro', 'cinco', 'z', 1, 0): 97,
#  ('cuatro', 'cinco', 'z', 1, 1): 75,
#  ('cuatro', 'cinco', 'z', 3, 0): 76,
#  ('cuatro', 'cinco', 'z', 3, 1): 53,
#  ('cuatro', 'cinco', 'z', 7, 0): 19,
#  ('cuatro', 'cinco', 'z', 7, 1): 49}

# P = {('uno', 'dos', 'p', 1): 1558,
#  ('uno', 'dos', 'p', 5): 406,
#  ('uno', 'dos', 'p', 7): 82,
#  ('uno', 'dos', 'z', 1): 1999,
#  ('uno', 'dos', 'z', 3): 1092,
#  ('uno', 'dos', 'z', 7): 481,
#  ('uno', 'tres', 'p', 1): 1339,
#  ('uno', 'tres', 'p', 5): 403,
#  ('uno', 'tres', 'p', 7): 228,
#  ('uno', 'tres', 'z', 1): 1749,
#  ('uno', 'tres', 'z', 3): 976,
#  ('uno', 'tres', 'z', 7): 56,
#  ('uno', 'cuatro', 'p', 1): 1710,
#  ('uno', 'cuatro', 'p', 5): 1436,
#  ('uno', 'cuatro', 'p', 7): 905,
#  ('uno', 'cuatro', 'z', 1): 1990,
#  ('uno', 'cuatro', 'z', 3): 601,
#  ('uno', 'cuatro', 'z', 7): 175,
#  ('uno', 'cinco', 'p', 1): 1228,
#  ('uno', 'cinco', 'p', 5): 785,
#  ('uno', 'cinco', 'p', 7): 738,
#  ('uno', 'cinco', 'z', 1): 1970,
#  ('uno', 'cinco', 'z', 3): 242,
#  ('uno', 'cinco', 'z', 7): 81,
#  ('dos', 'tres', 'p', 1): 1721,
#  ('dos', 'tres', 'p', 5): 119,
#  ('dos', 'tres', 'p', 7): 88,
#  ('dos', 'tres', 'z', 1): 1997,
#  ('dos', 'tres', 'z', 3): 1923,
#  ('dos', 'tres', 'z', 7): 1745,
#  ('dos', 'cuatro', 'p', 1): 1103,
#  ('dos', 'cuatro', 'p', 5): 926,
#  ('dos', 'cuatro', 'p', 7): 262,
#  ('dos', 'cuatro', 'z', 1): 1511,
#  ('dos', 'cuatro', 'z', 3): 1212,
#  ('dos', 'cuatro', 'z', 7): 1182,
#  ('dos', 'cinco', 'p', 1): 903,
#  ('dos', 'cinco', 'p', 5): 580,
#  ('dos', 'cinco', 'p', 7): 372,
#  ('dos', 'cinco', 'z', 1): 474,
#  ('dos', 'cinco', 'z', 3): 309,
#  ('dos', 'cinco', 'z', 7): 89,
#  ('tres', 'cuatro', 'p', 1): 1879,
#  ('tres', 'cuatro', 'p', 5): 1877,
#  ('tres', 'cuatro', 'p', 7): 105,
#  ('tres', 'cuatro', 'z', 1): 1424,
#  ('tres', 'cuatro', 'z', 3): 1394,
#  ('tres', 'cuatro', 'z', 7): 678,
#  ('tres', 'cinco', 'p', 1): 1326,
#  ('tres', 'cinco', 'p', 5): 691,
#  ('tres', 'cinco', 'p', 7): 312,
#  ('tres', 'cinco', 'z', 1): 967,
#  ('tres', 'cinco', 'z', 3): 952,
#  ('tres', 'cinco', 'z', 7): 851,
#  ('cuatro', 'cinco', 'p', 1): 1394,
#  ('cuatro', 'cinco', 'p', 5): 779,
#  ('cuatro', 'cinco', 'p', 7): 603,
#  ('cuatro', 'cinco', 'z', 1): 1133,
#  ('cuatro', 'cinco', 'z', 3): 876,
#  ('cuatro', 'cinco', 'z', 7): 504}



In [4]:
BR = {}
for i,j in NAD:
    listTemp = list(combinations(rota1[rota1.index(i):rota1.index(j)+1], 2))
    listTemp =  {(ii,jj) for ii,jj in listTemp if (ii,jj) in OD and (ii, jj) != (i,j)} #rota1.index(jj) == rota1.index(ii)+1 and 
    BR[(i,j)] = listTemp

# indices
index0 = [(i,j,v,k,t) for i in I2 for j in J for v in V for k in VK[v] for t in T]
index = [(i,j,v,k,t) for i in I for j in J for v in V for k in VK[v] for t in T]

In [5]:
model = Model("Modelo 1.1.1")

# variables de decicion
X = model.addVars(index0, vtype=GRB.INTEGER , name="X")
Y = model.addVars(index0, vtype=GRB.INTEGER , name="Y")
A = model.addVars(I2, vtype=GRB.INTEGER , name="A")
BNA = model.addVars(index, vtype=GRB.BINARY , name="BNA")
Z = model.addVars(index, vtype=GRB.BINARY , name="Z")

# funcion objetivo
model.setObjective(
    quicksum(P[(i,j,v,k)]*X[(i,j,v,k,t)] for i,j in OD for v in V for k in VK[v] for t in T), 
    sense = GRB.MAXIMIZE
)

# restricciones
for i in I:
    model.addConstr(
        A[rota[rota.index(i)]] == A[rota[rota.index(i)-1]] - quicksum(X[rota[rota.index(i)-1],j,v,k,t] for j in J for v in V for k in VK[v] for t in T if rota.index(j) >= rota.index(i)) + quicksum(X[j,i,v,k,t] for j in I for v in V for k in VK[v] for t in T if rota.index(j) < rota.index(i) and (j,i) in OD),
        name=f"Dispo_{i}"
    )

    model.addConstr(
        quicksum((X[i,j,v,k,t]) for j in J for v in V for k in VK[v] for t in T if i != j and (i,j) in OD) <= A[rota[rota.index(i)]],
        name=f"Cap_{i}"
    )

    model.addConstr(
        quicksum(Y[i,j,v,VK[v][0],t] for j in J for v in V for t in T if i != j and (i,j) in OD) <= Q, 
        name=f"AuthoCap_{i}"
    )


for i,j in OD:
    for v in V:
        for k in VK[v]:
            for t in T:

                model.addConstr(
                    X[i,j,v,k,t] <= d[i,j,v,k,t],
                    name = f"Assig_({i},{j},{v},{k},{t})"
                )

                if k != VK[v][-1]:
                    pos_k = VK[v].index(k)
                    model.addConstr(
                        Y[i,j,v,k,t] >= Y[i,j,v,VK[v][pos_k+1],t], 
                        name=f"Classe_({i},{j},{v},{k},{t})"
                    )

                    model.addConstr(
                        Y[i,j,v,k,t] >=  X[i,j,v,k,t] + Y[i,j,v,VK[v][pos_k+1],t],
                        name=f"Autho_({i},{j},{v},{k},{t})"
                    )
                else:
                    model.addConstr(
                        Y[i,j,v,k,t] >=  X[i,j,v,k,t],
                        name=f"Autho_({i},{j},{v},{k},{t})"
                    )

                

# [start] --- Restricciones de Capitalismo
for o,d in NAD:
    for v in V:
        for k in VK[v]:
            for t in T:

                model.addConstr(
                    BNA[o,d,v,k,t] <= Y[o,d,v,k,t],
                    name = f"activ_bin_autho_low_({o},{d},{v},{k},{t})"
                )

                model.addConstr(
                    Y[o,d,v,k,t] <= Q*BNA[o,d,v,k,t],
                    name = f"activ_bin_autho_top_({o},{d},{v},{k},{t})"
                )


    for i,j in BR[o,d]:
        for v in V:
            for k in VK[v]:
                for t in T:

                    lista = VK[v]
                    pos_k = lista.index(k)

                    if pos_k == 0:
                        model.addConstr(
                            BNA[o,d,v,k,t] - BNA[o,d,v,lista[pos_k+1],t] -1 <= quicksum(Y[i,j,v,kk,t] for kk in VK[v] if lista.index(kk) > lista.index(k)),
                            name = f"cal_betwen_autho_val_({o},{d},{i},{j},{v},{k},{t})"
                        )

                        model.addConstr(
                            quicksum(Y[i,j,v,kk,t] for kk in VK[v] if lista.index(kk) > lista.index(k)) <= -1*Q*(BNA[o,d,v,k,t] - BNA[o,d,v,lista[pos_k+1],t] -1),
                            name = f"cal_betwen_autho_val2_({o},{d},{i},{j},{v},{k},{t})"
                        )
                    elif pos_k == len(lista)-1:
                        model.addConstr(
                            BNA[o,d,v,k,t] <= Y[i,j,v,k,t],
                            name = f"cal_betwen_autho_val_({o},{d},{i},{j},{v},{k},{t})"
                        )

                        model.addConstr(
                            Y[i,j,v,k,t] <= Q*BNA[o,d,v,k,t],
                            name = f"cal_betwen_autho_val2_({o},{d},{i},{j},{v},{k},{t})"
                        )
                    elif len(lista) >= 3:
                        model.addConstr(
                            BNA[o,d,v,k,t] + BNA[o,d,v,lista[pos_k-1],t] - BNA[o,d,v,lista[pos_k+1],t] - 2 <= quicksum(Y[i,j,v,kk,t] for kk in VK[v] if lista.index(kk) > lista.index(k)),
                            name = f"cal_betwen_autho_val_({o},{d},{i},{j},{v},{k},{t})"
                        )

                        model.addConstr(
                            quicksum(Y[i,j,v,kk,t] for kk in VK[v] if lista.index(kk) > lista.index(k)) <= -1*Q*(BNA[o,d,v,k,t] + BNA[o,d,v,lista[pos_k-1],t] - BNA[o,d,v,lista[pos_k+1],t] - 2),
                            name = f"cal_betwen_autho_val2_({o},{d},{i},{j},{v},{k},{t})"
                        )
                    else:
                        pass
# [end] --- Restricciones de Capitalismo


for j in J:
    for v in V:
        for k in VK[v]:
            for t in T:
                model.addConstr(
                    X[0,j,v,k,t] == 0,
                    name = f"Assig_({0},{j},{v},{k},{t})"
                )

                model.addConstr(
                    Y[0,j,v,k,t] == 0,
                    name = f"Autho_({0},{j},{v},{k},{t})"
                )


model.addConstr(
    A[0] == Q,
    name = f"Cap_0"
)



Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-11


In [ ]:
# Optimizar el modelo
model.optimize()

In [ ]:
model.write('wilmer.lp')
model.display()